# Trade Data Cleaning and Preparation

In this notebook, I’ll clean and merge my trade export data for:
- Ayurvedic Herbs
- Millets
- Turmeric

I have two kinds of datasets:
- Monthly data (India as reporter)
- Annual data (Global overview)

Goal: Create one clean master file to use later for analysis and Power BI visualization.


In [38]:
import pandas as pd
import os

# File locations (update these as per your folder)
raw_path = r"C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\raw"
clean_path = r"C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned"

os.makedirs(clean_path, exist_ok=True)

print("Folders verified.")


Folders verified.


In [39]:
## Step 3: Create a Simple Cleaning Function

In [40]:
import pandas as pd

def clean_trade_file(df, product_name, origin):
    #1. Standardize column names
    df.columns = df.columns.str.strip().str.lower()

    #2. Keep only necessary columns (based on your dataset)
    keep = [
        "refyear", "refmonth", "reporterdesc",
        "flowdesc", "partnerdesc", "cmdcode",
        "qtyunitabbr", "qty", "netwgt", "primaryvalue"
    ]
    df = df[[c for c in keep if c in df.columns]]

    #3. Rename columns to a consistent schema
    df = df.rename(columns={
        "refyear": "year",
        "refmonth": "month",
        "reporterdesc": "reporter_country",
        "flowdesc": "trade_flow",
        "partnerdesc": "partner_country",
        "cmdcode": "hs_code",
        "qtyunitabbr": "unit",
        "qty": "quantity",
        "netwgt": "net_weight",
        "primaryvalue": "trade_value_usd"
    })

    #4. Convert numeric fields
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["month"] = pd.to_numeric(df["month"], errors="coerce")
    df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")
    df["net_weight"] = pd.to_numeric(df["net_weight"], errors="coerce")

    df["trade_value_usd"] = (
        df["trade_value_usd"]
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
    )
    df["trade_value_usd"] = pd.to_numeric(df["trade_value_usd"], errors="coerce")

    #5. Clean text fields
    for col in ["reporter_country", "partner_country", "trade_flow", "unit"]:
        df[col] = df[col].astype(str).str.title().str.strip()

    #6. Add custom columns
    df["product_name"] = product_name
    df["origin"] = origin

    return df

In [41]:
#Clean Monthly (India) Files

In [42]:
monthly_path = os.path.join(raw_path, "monthly")
files = os.listdir(monthly_path)
print("Total monthly files found:", len(files))

# === list all monthly files ===
all_files = [f for f in os.listdir(monthly_path) if f.endswith(".csv")]

# === group files by product ===
product_groups = {}
for f in all_files:
    product_name = f.split("_")[0].lower().strip()  # ayurvedicherbs, turmeric, millet
    product_groups.setdefault(product_name, []).append(f)

# === process each product group ===
for product, files in product_groups.items():
    output_file = os.path.join(clean_path, f"{product.title()}_India_Monthly_Clean.csv")

    # 🛑 Skip if file already exists
    if os.path.exists(output_file):
        print(f"⏩ Skipping {product.title()} — cleaned file already exists.")
        continue

    print(f"\n🧹 Cleaning monthly files for {product.title()}...")
    all_cleaned = []

    for f in sorted(files):
        file_path = os.path.join(monthly_path, f)
        print(f"   → Reading {f}")
        df_raw = pd.read_csv(file_path, encoding="latin1")
        df_clean = clean_trade_file(df_raw, product.title(), "India")
        all_cleaned.append(df_clean)

    # Combine all years
    df_final = pd.concat(all_cleaned, ignore_index=True)

    # Save combined cleaned file
    df_final.to_csv(output_file, index=False)
    print(f"✅ Saved: {output_file}")

print("\n🎉 All monthly datasets cleaned successfully!")



Total monthly files found: 15

🧹 Cleaning monthly files for Ayurvedicherbs...
   → Reading ayurvedicherbs_2020_monthly.csv
   → Reading ayurvedicherbs_2021_monthly.csv
   → Reading ayurvedicherbs_2022_monthly.csv
   → Reading ayurvedicherbs_2023_monthly.csv
   → Reading ayurvedicherbs_2024_monthly.csv
✅ Saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned\Ayurvedicherbs_India_Monthly_Clean.csv

🧹 Cleaning monthly files for Millet...
   → Reading Millet_2020_monthly.csv
   → Reading Millet_2021_monthly.csv
   → Reading Millet_2022_monthly.csv
   → Reading Millet_2023_monthly.csv
   → Reading Millet_2024_monthly.csv
✅ Saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned\Millet_India_Monthly_Clean.csv

🧹 Cleaning monthly files for Turmeric...
   → Reading turmeric_2020_monthly.csv
   → Reading turmeric_2021_monthly.csv
   → Reading turmeric_2022_monthly.csv
   → Reading turmeric_2023_monthly.csv
   → Reading turmeric_2024_mon

In [43]:
#Global cleaner script

In [44]:
global_path = os.path.join(raw_path, "Global")
files = os.listdir(global_path)
print("Total global files found:", len(files))

# === list all CSV files ===
all_files = [f for f in os.listdir(global_path) if f.endswith(".csv")]

# === group by product name (first part of filename) ===
product_groups = {}
for f in all_files:
    product_name = f.split("_")[0].lower().strip()
    product_groups.setdefault(product_name, []).append(f)

def clean_trade_file(df, product_name, origin):
    # Standardize column names
    df.columns = df.columns.str.strip().str.lower()

    # Keep only relevant columns that exist
    keep = [
        "refyear", "reporterdesc", "partnerdesc", "flowdesc",
        "primaryvalue", "qty", "qtyunitabbr", "cmdcode", "netwgt"
    ]
    existing_cols = [c for c in keep if c in df.columns]
    df = df[existing_cols]

    # Rename columns dynamically (only if they exist)
    rename_map = {
        "refyear": "year",
        "reporterdesc": "reporter_country",
        "partnerdesc": "partner_country",
        "flowdesc": "trade_flow",
        "primaryvalue": "trade_value_usd",
        "qty": "quantity",
        "qtyunitabbr": "unit",
        "cmdcode": "hs_code",
        "netwgt": "net_weight"
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Convert and clean numeric fields (only if present)
    if "year" in df.columns:
        df["year"] = pd.to_numeric(df["year"], errors="coerce")

    if "trade_value_usd" in df.columns:
        df["trade_value_usd"] = (
            df["trade_value_usd"].astype(str)
            .str.replace(",", "")
            .str.replace(" ", "")
        )
        df["trade_value_usd"] = pd.to_numeric(df["trade_value_usd"], errors="coerce")

    if "quantity" in df.columns:
        df["quantity"] = pd.to_numeric(df["quantity"], errors="coerce")

    if "net_weight" in df.columns:
        df["net_weight"] = pd.to_numeric(df["net_weight"], errors="coerce")

    # Clean text fields safely
    for col in ["reporter_country", "partner_country", "trade_flow", "hs_description"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.title().str.strip()

    # Add metadata
    df["product_name"] = product_name
    df["origin"] = origin

    return df

# === process each product ===
for product, files in product_groups.items():
    output_file = os.path.join(clean_path, f"{product.title()}_Global_Clean.csv")

    # 🛑 Skip if already exists
    if os.path.exists(output_file):
        print(f"⏩ Skipping {product.title()} — cleaned file already exists.")
        continue

    print(f"\n🌍 Cleaning Global files for {product.title()}...")
    all_cleaned = []

    for f in sorted(files):
        file_path = os.path.join(global_path, f)
        print(f"   → Reading {f}")
        df_raw = pd.read_csv(file_path, encoding="latin1")
        df_clean = clean_trade_file(df_raw, product.title(), "Global")
        all_cleaned.append(df_clean)

    df_final = pd.concat(all_cleaned, ignore_index=True)
    df_final.to_csv(output_file, index=False)
    print(f"✅ Saved: {output_file}")

print("\n🎉 All Global datasets cleaned successfully!")



Total global files found: 3

🌍 Cleaning Global files for Ayurvedicherbs...
   → Reading AyurvedicHerbs_2020-2024_Global.csv
✅ Saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned\Ayurvedicherbs_Global_Clean.csv

🌍 Cleaning Global files for Millet...
   → Reading Millet_2020-2024_Global.csv
✅ Saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned\Millet_Global_Clean.csv

🌍 Cleaning Global files for Turmeric...
   → Reading Turmeric_2020-2024_Global.csv
✅ Saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned\Turmeric_Global_Clean.csv

🎉 All Global datasets cleaned successfully!


In [45]:
#Combine Both into a Master File

In [46]:
processed_path = r"C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\processed"
os.makedirs(processed_path, exist_ok = True)
cleaned_files = os.listdir(clean_path)
cleaned_files


cleaned_folder = r"C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\cleaned"

# Suppose you already have a list of cleaned DataFrames
cleaned_files = []

for file in os.listdir(cleaned_folder):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(cleaned_folder, file))
        cleaned_files.append(df)

# Combine all cleaned DataFrames
master = pd.concat(cleaned_files, ignore_index=True)

# Preview combined dataset
print(master.head(3))
print(" Combined dataset shape:", master.shape)


   year reporter_country partner_country trade_flow  trade_value_usd  hs_code  \
0  2020          Albania           World     Export     3.927840e+07   121190   
1  2020           Angola           World     Export     6.533650e+02   121190   
2  2020       Azerbaijan           World     Export     1.455131e+06   121190   

    net_weight    product_name  origin  month unit  quantity  
0  11187975.92  Ayurvedicherbs  Global    NaN  NaN       NaN  
1       106.03  Ayurvedicherbs  Global    NaN  NaN       NaN  
2   4288809.00  Ayurvedicherbs  Global    NaN  NaN       NaN  
 Combined dataset shape: (15061, 12)


In [47]:
# Save final master dataset
master_output = os.path.join(processed_path, "Trade_Master.csv")
master.to_csv(master_output, index=False)

print("✅ Final Master File saved:", master_output)

✅ Final Master File saved: C:\Users\hp\Downloads\Project - export\New\Day 2- Data Gathering\data\processed\Trade_Master.csv


In [48]:
master["product_name"]=master["product_name"].replace("Ayurvadicherbs","Ayurvedicherbs")


In [49]:
print(master["product_name"].value_counts())
print(master["origin"].value_counts())
print(master["year"].unique())

product_name
Turmeric          6318
Ayurvedicherbs    6200
Millet            2543
Name: count, dtype: int64
origin
India     13431
Global     1630
Name: count, dtype: int64
[2020 2021 2022 2023 2024]


In [50]:
master.head(10)

,year,reporter_country,partner_country,trade_flow,trade_value_usd,hs_code,net_weight,product_name,origin,month,unit,quantity
0,2020,Albania,World,Export,3.927840e+07,121190,1.118798e+07,Ayurvedicherbs,Global,NaN,NaN,NaN
1,2020,Angola,World,Export,6.533650e+02,121190,1.060300e+02,Ayurvedicherbs,Global,NaN,NaN,NaN
2,2020,Azerbaijan,World,Export,1.455131e+06,121190,4.288809e+06,Ayurvedicherbs,Global,NaN,NaN,NaN
3,2020,Argentina,World,Export,1.270719e+06,121190,5.391682e+05,Ayurvedicherbs,Global,NaN,NaN,NaN
4,2020,Australia,World,Export,1.659831e+07,121190,9.476728e+05,Ayurvedicherbs,Global,NaN,NaN,NaN
5,2020,Austria,World,Export,5.540101e+07,121190,9.610282e+06,Ayurvedicherbs,Global,NaN,NaN,NaN
6,2020,Bahrain,World,Export,2.289710e+06,121190,4.358221e+04,Ayurvedicherbs,Global,NaN,NaN,NaN
7,2020,Armenia,World,Export,4.964061e+05,121190,2.033664e+05,Ayurvedicherbs,Global,NaN,NaN,NaN
8,2020,Barbados,World,Export,3.388000e+03,121190,2.600000e+01,Ayurvedicherbs,Global,NaN,NaN,NaN
9,2020,Belgium,World,Export,7.883548e+06,121190,1.374441e+06,Ayurvedicherbs,Global,NaN,NaN,NaN
